In [ ]:
from numpy import *
import astropy.units as u
from astropy.constants import G, M_jup, R_jup, M_earth, R_earth, L_sun, M_sun, R_sun
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import extinction as ex
import scipy.stats as st

In [ ]:
#import scaling relation coefficients
rel = pd.read_csv('scalingrels_a17.csv')
rel.set_index('Tracer', inplace=True)

## Functions:

Cell 1: Line luminosity (additional ones in Test notebook)

Cell 2: Accretion luminosity. Has error analysis capability built in, but is not utilized right now.

Cell 3: Accretion rate

Cell 4: Get rate (run through 1, 2, and 3 sequentially)


In [ ]:
def line_lum(line_flux, dist):
    """
    Calculate line luminosity given line flux and distance
    assuming line flux is extinction corrected.
    """
    #future: could add extinction correction in there? Not sure how that works.
    
    line_lum = 4 * pi * (dist*u.pc)**2 * line_flux * u.erg / (u.s * (u.cm)**2)
    line_lum = line_lum.decompose().to(u.W)
    return line_lum/u.W

In [ ]:
def accr_lum(L_line, tracer, L_line_err = 0*u.W):
    """
    Translate a line luminosity to accretion luminosity using empirical
    relationships from Alcala et al. 2017.
    
    Included tracers are:
    'Ha'
    'Hb'
    'Hg'
    'PaB'
    'PaG'
    'He I 587.6'
    'He I 667.8'
    'Ca II K'
    'Ca II H'
    """
        
    a, a_err, b, b_err = rel['a'][tracer],rel['a_err'][tracer],rel['b'][tracer],rel['b_err'][tracer]
    
    log_L_acc = b + a * log10(L_line*u.W/L_sun)
    
    L_acc = 10**log_L_acc*L_sun/u.W
    
    #attempt at error propagation
    
    #c_err = (L_line_err)/(log(10) * L_line)
    #ac_err = a * log10(L_line/L_sun) * ((a_err/a)**2 + (c_err/log10(L_line/L_sun))**2)**0.5
    #log_L_acc_err = (b_err**2 + ac_err**2)**0.5
    #L_acc_err = L_acc * log(10) * log_L_acc_err

    return L_acc

In [ ]:
def acc_rate(L_acc, R, M):
    """
    Translate an accretion luminosity and planet mass/radius to accretion rate in Solar masses per year.
    """
    mdot = 1.25*L_acc*u.W*R*u.R_sun/(G*M*u.M_sun)
    mdot = mdot.decompose().to(u.M_sun/u.yr)
    return(mdot/(u.M_sun/u.yr))

In [ ]:
def get_rate(line_flux, d, t, R, M):
    """
    Turn a line flux into an accretion rate, 
    given the distance, tracer, object radius, and object mass.
    
    Line flux should be in erg/s/cm^2; distance in pc;
    tracer either Ha, PaB, or BrG; radius in solar radii,
    and mass in solar masses.
    """
    l_line = line_lum(line_flux,d)
    l_acc = accr_lum(l_line, t)
    mdot = acc_rate(l_acc, R, M)
    
    return mdot

### Recalculate accretion rates for Ha, PaB, and BrG tracers

Using new distances but old masses

In [ ]:
db = pd.read_csv('recalculations.csv')
tracers = ['Ha','PaB','BrG']

In [ ]:
for i in db.index:
    
    #use new distance if it's available
    #else, use arbitrary distance
    #(some are legitimately pulled from papers but other just assume 140)
    d = db.loc[i,'Plx Distance']
    if isnan(d):
        d = db.loc[i,'Object Distance']
    
    R = db.loc[i,'Object Radius']
    M = db.loc[i,'Object Mass, Original']
    
    #ignore if missing M or R
    if (isnan(M) or isnan(R)):
        continue
        
    for t in tracers:
        lf = db.loc[i,t + ' Line Flux']
        
        if isnan(lf)==False:
            
            #extinction correction for N06 fluxes
            if db.loc[i,'Reference']=='Natta 2006':
                lf = ex.remove(db.loc[i,'A_J'],lf)
            
            #get the accretion rate
            db.at[i,t + ' Accr Rate'] = get_rate(lf, d, t, R, M)

In [ ]:
#take average accretion rate across the three tracers
db['Avg Accr Rate'] = db[[t + ' Accr Rate' for t in tracers]].mean(axis=1)

Now, switch to plotting and analysis

In [ ]:
#Take relevant columns and drop NaNs
aa = db[['Source','Reference','Object Mass, Original', 'Avg Accr Rate','Old Accr Rate']].dropna()

In [ ]:
#lists of mass, old accr rate, and new accr rate
m = log10(aa['Object Mass, Original'].tolist())
md_old = log10(aa['Old Accr Rate'].tolist())
md_new = log10(aa['Avg Accr Rate'].tolist())

In [ ]:
sl_old, incpt_old = polyfit(m,md_old,1)
sl_new, incpt_new = polyfit(m,md_new,1)
print('Old Best-Fit: y = ', sl_old,'x ', incpt_old)
print('New Best-Fit: y = ', sl_new,'x ', incpt_new)

In [ ]:
#correlations
r_old = st.pearsonr(m,md_old)
r_new = st.pearsonr(m,md_new)
print('Old r:', r_old)
print('New r:', r_new)

In [ ]:
print('Number of points included:', len(aa))

In [ ]:
plt.figure(figsize=(10,10/1.625),dpi=400)

plt.scatter(m,md_old,color='r',alpha=0.6,label='r=0.70')
plt.scatter(m,md_new,color='g',alpha=0.6,label='r=0.67')

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5)
plt.ylim(-14,-6)

plt.plot(x, sl_old*x+incpt_old, color='r', label='y=1.90x-8.07')
plt.plot(x, sl_new*x+incpt_new, color='g', label='y=1.64x-8.23')

plt.legend(frameon=False)

plt.xlabel('log $M$  $(M_{\odot})$')
plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

Above: Plot of $M$ vs $\dot M$ for originally estimated accretion rates (red) and new estimates (green). New estimates are averages of those obtained by Ha, PaB, and BrG tracers. The best-fit line for each population is overplotted in its respective color.

Notice the correlation decreases slightly for the new estimates; this may be due to the lack of mass updates. Let's find out.

## Recalculate accretion rates: new masses!

Note that I am still using old radii

In [ ]:
db2 = pd.read_csv('recalculations.csv')

In [ ]:
#get accretion rates

#mass_used: one list of which mass is used to recalculate accretion rate
mass_used = []

for i in db2.index:
    
    #use new distance if it's available
    #else, use arbitrary distance
    #(some are legitimately pulled from papers but others just assume 140)
    
    d = db2.loc[i,'Plx Distance']
    if isnan(d):
        d = db2.loc[i,'Object Distance']
    
    R = db2.loc[i,'Object Radius']
    
    M = db2.loc[i,'New Mass']
    if isnan(M):
        M = db2.loc[i,'Object Mass, Original']
        
    mass_used.append(M)

    #ignore if missing M or R
    if (isnan(M) or isnan(R)):
        continue
            
    for t in tracers:
        lf = db2.loc[i,t + ' Line Flux']
        
        if isnan(lf)==False:
            
            #extinction correction for N06 fluxes
            if db2.loc[i,'Reference']=='Natta 2006':
                lf = ex.remove(db2.loc[i,'A_J'],lf)
            
            #get the accretion rate
            db2.at[i,t + ' Accr Rate'] = get_rate(lf, d, t, R, M)

In [ ]:
db2['mass_used'] = mass_used

In [ ]:
db2['Avg Accr Rate'] = db2[[t + ' Accr Rate' for t in tracers]].mean(axis=1)

In [ ]:
#Take relevant columns and drop NaNs
aa2 = db2[['Source','Reference','Object Mass, Original','mass_used',
           'Avg Accr Rate','Old Accr Rate']].dropna()

In [ ]:
#lists of mass, old accr rate, and new accr rate
m_old = log10(aa2['Object Mass, Original'].tolist())
m_new = log10(aa2['mass_used'].tolist())
md_old2 = log10(aa2['Old Accr Rate'].tolist())
md_new2 = log10(aa2['Avg Accr Rate'].tolist())

In [ ]:
sl_old2, incpt_old2 = polyfit(m_old,md_old2,1)
sl_new2, incpt_new2 = polyfit(m_new,md_new2,1)
print('Old Best-Fit: y = ', sl_old2,'x ', incpt_old2)
print('New Best-Fit: y = ', sl_new2,'x ', incpt_new2)

In [ ]:
#correlations
r_old2 = st.pearsonr(m_old,md_old2)
r_new2 = st.pearsonr(m_new,md_new2)
print('Old r:', r_old2)
print('New r:', r_new2)

In [ ]:
print('Number of rows included:', len(aa2))
print('This is not the number of points, as there are duplicate rows that have not been corrected in the database')

In [ ]:
plt.figure(figsize=(10,10/1.625),dpi=400)

plt.scatter(m_old,md_old2,color='r',alpha=0.6,label='r=0.70')
plt.scatter(m_new,md_new2,color='g',alpha=0.6,label='r=0.65')

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5)
plt.ylim(-14,-6)

plt.plot(x, sl_old2*x+incpt_old2, color='r', label='y=1.90x-8.07')
plt.plot(x, sl_new2*x+incpt_new2, color='g', label='y=1.64x-8.23')

plt.legend(frameon=False)

plt.xlabel('log $M$  $(M_{\odot})$')
plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

Above: Plot of $M$ vs $\dot M$ for originally estimated masses and accretion rates (red) and new estimates for masses and accretion rates (green). New estimates are averages of those obtained by Ha, PaB, and BrG tracers. The best-fit line for each population is overplotted in its respective color.

In [ ]:
plt.figure(figsize=(10,10/1.625),dpi=400)

plt.scatter(m_old,md_old2,color='r',alpha=0.6,label='r=0.70')
plt.scatter(m_new,md_new2,color='g',alpha=0.6,label='r=0.65')

x = arange(-2.5,1,0.5)
plt.xlim(-2.5,0.5)
plt.ylim(-14,-6)

plt.plot(x, sl_old2*x+incpt_old2, color='r', label='y=1.90x-8.07')
plt.plot(x, sl_new2*x+incpt_new2, color='g', label='y=1.64x-8.23')

plt.legend(frameon=False)

for i in range(len(m_old)):
    plt.plot([m_old[i],m_new[i]],[md_old2[i],md_new2[i]],color='k',linestyle='dotted',alpha=0.35)


plt.xlabel('log $M$  $(M_{\odot})$')
plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')

Same plot as above, but with old and new estimates connected by gray dotted lines.

### Connect points corresponding to same source
And flag ones with different masses

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
d = aa2.sort_values(by='Source')
d

In [ ]:
dm = log10(d['mass_used'].tolist())
ddm = log10(d['Avg Accr Rate'].tolist())

In [ ]:
for i in range(len(d)-1):
    if d.loc[d.index[i],'Source']==d.loc[d.index[i+1],'Source']:
        if abs(dm[i]-dm[i+1]) >0:
            print('The sources with these indices match:',d.index[i],d.index[i+1],
                  '\n Masses: ', round(10**dm[i],3),'and',round(10**dm[i+1],3),
                  '\n', round(dm[i]-dm[i+1],2), 'orders of magnitude apart. \n')

In [ ]:
plt.figure(figsize=(10,10/1.625),dpi=400)

plt.scatter(dm,ddm)

plt.xlim(-2.5,0.5)
plt.ylim(-14,-6)

plt.plot(x, sl_new2*x+incpt_new2, color='b', label='y=1.67x-8.22')

for i in range(len(d)-1):
    if d.loc[d.index[i],'Source']==d.loc[d.index[i+1],'Source']:
        plt.plot([dm[i],dm[i+1]],[ddm[i],ddm[i+1]],
                 color='k',linestyle='dotted',alpha=0.35)

plt.xlabel('log $M$  $(M_{\odot})$')
plt.ylabel('log $\dot M$  $(M_{\odot}/yr)$')